In [1]:
import sys
sys.path.insert(0,r"C:\Users\pc\Documents")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Lectura de datos     
base_sur = pd.read_csv(r'C:\Users\pc\Downloads\BASE_SUR.txt', sep=r'¡', engine='python')
base_sur.head()

## Eliminación de variables: 
base_sur2 = base_sur[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO',
        'PROM_SUSC', 'CANAL_ENVIO', 'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

In [3]:
# Vista previa
base_sur2.head()

,SESUSERV,SERVDESC,SESUSUSC,SESUFEIN,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,SUSCNITC,TIPIFICACION_CLIENTE,...,CANTIDAD,CALIFICACION_VECTOR_SERVICIO,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,REGIONAL,ESTRATO_AGRUPADO,DEPARTAMENTO_AGR,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7255,TELEFONÍA,17027953,24/12/2019,CAUCA,1,27255,7,1062075580,NUEVO,...,3,92,"29442,67",DIGITAL,Hogares,SUR,2.0,CAUCA,151,FISICO
1,7255,TELEFONÍA,17034239,24/12/2019,VALLE DEL CAUCA,0,0,7,545248,RETIRADO,...,0,0,27494,DIGITAL,Hogares,SUR,1.0,VALLE DEL CAUCA,271,FISICO
2,7255,TELEFONÍA,17060337,09/01/2020,VALLE DEL CAUCA,1,10144,7,91457507,NUEVO,...,2,100,"25694,58",DIGITAL,Hogares,SUR,3.0,VALLE DEL CAUCA,131,FISICO
3,7245,INTERNET,17030650,22/12/2019,NARIÑO,1,71275,7,1085259877,NUEVO,...,4,88,"81373,2",DIGITAL,Hogares,SUR,5.0,NARIÑO,149,DIGITAL
4,7245,INTERNET,17210446,30/12/2019,VALLE DEL CAUCA,3,111476,7,38680889,NUEVO,...,3,25,"19144,56",FISICO,Hogares,SUR,2.0,VALLE DEL CAUCA,141,SIN PAGO


In [4]:
## Numero completo de clientes en la region
len(base_sur2)

393242

In [5]:
## Eliminacion de clientes con datos NA
base_sur3 = base_sur2.dropna()

In [6]:
len(base_sur3)

333077

In [7]:
## Group by por nit para calificacion
b4 = base_sur3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_sur4 = pd.merge(base_sur3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_sur4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_sur4['y'] = pd.cut(x=base_sur4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])


In [9]:
dataTypeSeries = base_sur4.dtypes
dataTypeSeries

cat = base_sur4.select_dtypes(include=['object','category'])
cuanti = base_sur4.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))

In [10]:
cuanti.corr(method = 'pearson')

,SESUSERV,SESUSUSC,SESUCUSA,SESUCICL,SUMA,CANTIDAD,CALIFICACION_VECTOR_SERVICIO_x,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CALIFICACION_VECTOR_SERVICIO_y
SESUSERV,1.000000,0.826559,0.103301,0.489386,0.311789,0.380274,0.383041,0.074453,0.050271,0.512404
SESUSUSC,0.826559,1.000000,0.172416,0.284997,0.234865,0.314546,0.403848,0.054595,-0.052742,0.470078
SESUCUSA,0.103301,0.172416,1.000000,0.009442,-0.162518,0.062890,-0.151704,-0.057054,-0.004050,-0.241468
SESUCICL,0.489386,0.284997,0.009442,1.000000,0.572131,0.608965,0.419723,-0.082434,0.235241,0.481513
SUMA,0.311789,0.234865,-0.162518,0.572131,1.000000,0.938040,0.826978,0.029437,0.073404,0.719855
CANTIDAD,0.380274,0.314546,0.062890,0.608965,0.938040,1.000000,0.739211,0.019181,0.090832,0.617349
CALIFICACION_VECTOR_SERVICIO_x,0.383041,0.403848,-0.151704,0.419723,0.826978,0.739211,1.000000,0.042642,0.036776,0.859830
ESTRATO_AGRUPADO,0.074453,0.054595,-0.057054,-0.082434,0.029437,0.019181,0.042642,1.000000,-0.380221,0.049594
ANTIGUEDAD_DIAS,0.050271,-0.052742,-0.004050,0.235241,0.073404,0.090832,0.036776,-0.380221,1.000000,0.042771
CALIFICACION_VECTOR_SERVICIO_y,0.512404,0.470078,-0.241468,0.481513,0.719855,0.617349,0.859830,0.049594,0.042771,1.000000


In [11]:
base_sur5=cuanti.join(cat)
print(base_sur5.dtypes)

#base_sur5.to_csv(r'C:\Users\pc\Desktop\base_sur_process.csv')

#base_sur5 = pd.read_csv(r'C:\Users\pc\Desktop\base_sur_process.csv', sep=',', engine='python')

X = base_sur5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 
        #'SESUSUSC',
        'DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12',
        #'REGIONAL',
        'PROM_SUSC', 'CANAL_ENVIO',
        # 'SUMA',
        #'CANTIDAD',
        #'CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y','DEPARTAMENTO_AGR'
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

y = base_sur5['y']

SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SESUSAPE                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
PROM_SUSC                           int32
CANAL_ENVIO                         int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [12]:
## Vista previa de las variables preliminares para clasificacion
X.head()

,SESUSERV,SERVDESC,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,TIPIFICACION_CLIENTE,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7255,20,1,1,20603,7,1,22193,0,1,2.0,151,1
1,7255,20,4,0,0,7,3,17780,0,1,1.0,271,1
2,7255,20,4,1,431,7,1,14700,0,1,3.0,131,1
3,7245,7,3,1,38589,7,1,108182,0,1,5.0,149,0
4,7245,7,4,3,3358,7,1,8363,1,1,2.0,141,3


In [13]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.893513  0.906240  0.899831  21203.0  21505.0  0.964784
1             0.842223  0.868571  0.855194  29088.0  29998.0  0.920656
2             0.690597  0.639326  0.663974  16325.0  15113.0  0.844477
avg / total   0.821390  0.824382  0.822541  66616.0  66616.0  0.922142
DecisionTreeClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.846575  0.927227  0.885067  21203.0  23223.0  0.964462
1             0.884906  0.792698  0.836268  29088.0  26057.0  0.932768
2             0.626961  0.665789  0.645792  16325.0  17336.0  0.853944
avg / total   0.809493  0.804416  0.805122  66616.0  66616.0  0.935491
RandomForestClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.886715  0.816583  0.850205  21203.0  19526.0  0.9

In [29]:
# Importancia de variables
model = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)

importance = model.feature_importances_
# Total importancia de variables
for i,v in enumerate(importance):
	print('Variable: %0d, Score: %.5f' % (i,v))

Variable: 0, Score: 0.00134
Variable: 1, Score: 0.00000
Variable: 2, Score: 0.00000
Variable: 3, Score: 0.43030
Variable: 4, Score: 0.10111
Variable: 5, Score: 0.03169
Variable: 6, Score: 0.15977
Variable: 7, Score: 0.00015
Variable: 8, Score: 0.00000
Variable: 9, Score: 0.00002
Variable: 10, Score: 0.00000
Variable: 11, Score: 0.02775
Variable: 12, Score: 0.00000
Variable: 13, Score: 0.11718
Variable: 14, Score: 0.13068


In [ ]:
## El mejor modelo de clasificacion fue el KNN, veamos el k optimo con corridas de k = 2 a k = 10

In [14]:
classifiers = [
    KNeighborsClassifier(2),
    KNeighborsClassifier(3),
    KNeighborsClassifier(4),
    KNeighborsClassifier(5),
    KNeighborsClassifier(6),
    KNeighborsClassifier(7),
    KNeighborsClassifier(8),
    KNeighborsClassifier(9),
    KNeighborsClassifier(10)
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.855947  0.940763  0.896353  21203.0  23304.0  0.955008
1             0.792202  0.909447  0.846785  29088.0  33393.0  0.903668
2             0.755419  0.458989  0.571026  16325.0   9919.0  0.820077
avg / total   0.803477  0.809025  0.794984  66616.0  66616.0  0.904914
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.893513  0.906240  0.899831  21203.0  21505.0  0.964784
1             0.842223  0.868571  0.855194  29088.0  29998.0  0.920656
2             0.690597  0.639326  0.663974  16325.0  15113.0  0.844477
avg / total   0.821390  0.824382  0.822541  66616.0  66616.0  0.922142
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.893892  0.926944  0.910118  21203.0  21987.0  0.96899

In [15]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
X_ = StandardScaler().fit_transform(X_fin)

for train_index, test_index in kf.split(X_, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(7).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                      y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


[             precision    recall  f1-score  support     pred       AUC
0             0.858386  0.853220  0.855796  21270.0  21142.0  0.943718
1             0.796134  0.837278  0.816188  29025.0  30525.0  0.910151
2             0.593445  0.543561  0.567409  16322.0  14950.0  0.803711
avg / total   0.766349  0.770404  0.767880  66617.0  66617.0  0.902720,              precision    recall  f1-score  support     pred       AUC
0             0.839674  0.871650  0.855363  21270.0  22080.0  0.944162
1             0.798720  0.820982  0.809698  29025.0  29834.0  0.902777
2             0.603523  0.543655  0.572027  16321.0  14702.0  0.810444
avg / total   0.763973  0.769215  0.766049  66616.0  66616.0  0.901168,              precision    recall  f1-score  support     pred       AUC
0             0.854144  0.864827  0.859452  21269.0  21535.0  0.946895
1             0.774381  0.829182  0.800845  29025.0  31079.0  0.885085
2             0.583244  0.500337  0.538619  16321.0  14001.0  0.767206
avg

In [ ]:
## Matriz de Confusion

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[18358,  1117,  1794],
       [  864, 24113,  4047],
       [ 2473,  4901,  8947]], dtype=int64)